In [50]:
import tensorflow as tf

In [51]:
import numpy as np
import cv2
import math
from PIL import Image, ImageStat
import sys
from os import listdir

In [52]:
def convert(dir):
    file_list=listdir(dir)
    for filename in file_list:
        print (filename)
        if "panorama" in filename and "cropped" not in filename:
            path=""
            path=dir+filename
            im=Image.open(path)
            out=im.resize((416,208),Image.ANTIALIAS)
            print ("%s has been resized"%filename)
            out.save(path)

In [53]:
def get_image(image_path):
    """
    Get a numpy 2D array of an image so that one can access RGBA[x][y] or RGBA[x,y].
    
    Also get a cropped image for further use for hsv and k-means clustering
    
    """
    origin = Image.open(image_path, 'r')
    width, height = origin.size
    area = (0, 0, width, 0.5*height)
    image = origin.crop(area) # crop top half of the image
#     result = Image.fromarray(image.astype(np.uint8))
    image.save(file+"_cropped.png")
    width, height = image.size
#     print (image.size)
    pixel_values = list(image.getdata())
    if image.mode == 'RGBA':
        channels = 4
    elif image.mode == 'L':
        channels = 1
    else:
        print("Unknown mode: %s" % image.mode)
        return None
#     pixel_values = np.array(pixel_values).reshape((width, height, channels))
    pixel_values = np.array(pixel_values).reshape((height, width, channels))
#     print (np.shape(pixel_values))

    return pixel_values,image,origin #return pixels matrix with RGBA value, and cropped half image

In [54]:
def get_bri(pixels):
    """
    Get a numpy 2D array of an image so that one can access brightness[x][y]
       
    """
    brightness=[[0 for x in range(len(pixels[0]))] for y in range(len(pixels))]
    # Matrix = [[0 for x in range(w)] for y in range(h)]
    for i in range(len(pixels)):
        for j in range(len(pixels[0])):
            R,G,B,A=pixels[i,j]
            brightness[i][j]=(0.2126*R + 0.7152*G + 0.0722*B)/255
    return brightness

In [55]:
path1 = "/Users/yimingqiu/Desktop/class/"
files= listdir(path1)

convert(path1)
# Sgray=0.0
# for K in range(4, 16):
# while Sgray<=1:
#     true_po_list=[]
#     precision_list=[]
#     edge_list=[]
    # print (files)


blackC-1010_-37.808706898558945_144.98444713162672_panorama.png_cropped.png
bigS-1060_-37.80868577546702_144.9850156495497_panorama.png
bigS-1060_-37.80868577546702_144.9850156495497_panorama.png has been resized
blackC-1010_-37.808706898558945_144.98444713162672_panorama.png
blackC-1010_-37.808706898558945_144.98444713162672_panorama.png has been resized
whiteB-0100_-37.80909086146474_144.9741000486449_panorama.png
whiteB-0100_-37.80909086146474_144.9741000486449_panorama.png has been resized
sky-1270_-37.80859702864577_144.98740342127365_panorama.png_cropped.png
sky-1810_-37.80836860123093_144.99354337933758_panorama.png
sky-1810_-37.80836860123093_144.99354337933758_panorama.png has been resized
bigS-2290_-37.80816528711204_144.99900108792806_panorama.png
bigS-2290_-37.80816528711204_144.99900108792806_panorama.png has been resized
tree-0110_-37.809086647010126_144.97421375343805_panorama.png_cropped.png
MNIST_data
buliding-0140_-37.80907400299053_144.9745548677395_panorama.png_crop

In [56]:
i=0
train_x = []
for file in sorted(files):
    if "panorama" in file and "cropped" not in file:
#         file=path1+file
        pixels,image,origin=get_image(file)
#         print (np.shape(pixels))
            # print (pixel1)
        brightness=get_bri(pixels)
        
#             edge_cal(file)
        brightness[i]=np.asarray(brightness)
        brightness[i]=brightness[i].reshape(-1)
        train_x.append(brightness[i])
        print(file,np.shape(train_x))
        i+=1
#         mark_sky(pixels,label2,origin,file,Sgray)
print(brightness[0][0:100])

bigS-0790_-37.80879980770527_144.98194564890238_panorama.png (1, 43264)
bigS-0820_-37.80878714139091_144.98228676055362_panorama.png (2, 43264)
bigS-0860_-37.80877025144173_144.98274157590654_panorama.png (3, 43264)
bigS-1060_-37.80868577546702_144.9850156495497_panorama.png (4, 43264)
bigS-2290_-37.80816528711204_144.99900108792806_panorama.png (5, 43264)
blackC-0850_-37.808774474092964_144.98262787208782_panorama.png (6, 43264)
blackC-0970_-37.80872379506533_144.98399231705417_panorama.png (7, 43264)
blackC-1010_-37.808706898558945_144.98444713162672_panorama.png (8, 43264)
blackC-1630_-37.80844477911086_144.99149673087072_panorama.png (9, 43264)
blackC-2520_-37.8080677765338_145.00161622930622_panorama.png (10, 43264)
buliding-0140_-37.80907400299053_144.9745548677395_panorama.png (11, 43264)
buliding-0380_-37.80897281542424_144.97728377794476_panorama.png (12, 43264)
buliding-1760_-37.808389765304874_144.99297486629814_panorama.png (13, 43264)
sky-1100_-37.80866887502632_144.985470

In [57]:
train_y = []
for i in range(5):
    train_y.append([1,0,0,0,0,0])
for i in range(5):
    train_y.append([0,1,0,0,0,0])
for i in range(3):
    train_y.append([0,0,1,0,0,0])
for i in range(5):
    train_y.append([0,0,0,1,0,0])
for i in range(5):
    train_y.append([0,0,0,0,1,0])
for i in range(5):
    train_y.append([0,0,0,0,0,1])

print(np.shape(train_y))

(28, 6)


In [58]:
test_x = train_x[0]
test_y = train_y[0]
train_x=np.delete(train_x,1,0)
train_y=np.delete(train_y,1,0)
print(np.shape(train_x),np.shape(train_y))

(27, 43264) (27, 6)


In [66]:
test_x=test_x.reshape((1,-1))
print(np.shape(test_x))

(1, 43264)


In [70]:
test_y= np.array(test_y)
test_y=test_y.reshape((1,-1))
print(np.shape(test_y))

(1, 6)


In [71]:

# batch_size = 100

# n_batch = mnist.train.num_examples // batch_size

batch_size=1

#placeholder
x = tf.placeholder(tf.float32,[None,43264])
y = tf.placeholder(tf.float32,[None,6])

#simple NN
W = tf.Variable(tf.zeros([43264,6]))
b = tf.Variable(tf.zeros([6]))
prediction = tf.nn.softmax(tf.matmul(x,W)+b)


loss = tf.reduce_mean(tf.square(y-prediction))
#gredient decent
train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)


init = tf.global_variables_initializer()

#save results
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))#argmax返回一维张量中最大的值所在的位置

accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs,batch_ys =  train_x,train_y
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys})
        
        acc = sess.run(accuracy,feed_dict={x:test_x,y:test_y})
        print("Iter " + str(epoch) + ",Testing Accuracy " + str(acc))

Iter 0,Testing Accuracy 0.0
Iter 1,Testing Accuracy 0.0
Iter 2,Testing Accuracy 0.0
Iter 3,Testing Accuracy 0.0
Iter 4,Testing Accuracy 0.0
Iter 5,Testing Accuracy 0.0
Iter 6,Testing Accuracy 0.0
Iter 7,Testing Accuracy 0.0
Iter 8,Testing Accuracy 0.0
Iter 9,Testing Accuracy 0.0
Iter 10,Testing Accuracy 0.0
Iter 11,Testing Accuracy 0.0
Iter 12,Testing Accuracy 0.0
Iter 13,Testing Accuracy 0.0
Iter 14,Testing Accuracy 0.0
Iter 15,Testing Accuracy 0.0
Iter 16,Testing Accuracy 0.0
Iter 17,Testing Accuracy 0.0
Iter 18,Testing Accuracy 0.0
Iter 19,Testing Accuracy 0.0
Iter 20,Testing Accuracy 0.0
